In [3]:
import os

In [4]:
%pwd

'c:\\Users\\yennu\\Desktop\\ETE_NLP_Text_Summarization\\research'

In [5]:
os.chdir('../')

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path




In [7]:
from src.constants import *
from src.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config

In [9]:
import os 
import urllib.request as request
import zipfile
from src.logging import logger
from src.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def Download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f'{filename} download! with following info: \n{headers}')
        else:
            logger.info(f'file already exists with size: {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.Download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-02 15:52:17,418: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-02 15:52:17,423: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-02 15:52:17,428: INFO: common: created directory at artifacts]
[2024-04-02 15:52:17,433: INFO: common: created directory at artifacts/data_ingestion]


[2024-04-02 15:53:48,283: INFO: 1218894149: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2CDC:28BC7D:733360:78DC0A:660BDCA6
Accept-Ranges: bytes
Date: Tue, 02 Apr 2024 10:23:35 GMT
Via: 1.1 varnish
X-Served-By: cache-fra-eddf8230035-FRA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1712053415.992182,VS0,VE553
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: a64d1595add76343322736d56343815cc2b7c8cb
Expires: Tue, 02 Apr 2024 10:28:35 GMT
Source-Age: 0

]
